In [1]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from AnsatzLacs import *
from CCAUtil import *
from qiskit_algorithms.optimizers import SPSA
import sys

In [2]:
# implementation of the slack variabel method
# quantum circuit settings

# these values will be used as global variables later in different functions
sys_size = 2 # size of the input state (2 means 2 qubit)
ref_size = 0 # size of the reference state (for purification)

num_layers = 2
qcbm_layers = 2

In [3]:
H_paulis = ["ZZ", "IX", "XI"]
coeff = [1, 1, 1]
As = ["YI", "IZ"]
H_unitary = [get_expectation_unitary(hi) for hi in H_paulis] 
As_unitary = [get_expectation_unitary(ai) for ai in As]
b = [0.2, 0.1]
INF = sys.maxsize

In [4]:
expectation_unitary_dict = {pauli_string: get_expectation_unitary(pauli_string) for pauli_string in H_paulis}
for ai in As:
    expectation_unitary_dict[ai] = get_expectation_unitary(ai)

In [5]:
def get_exp(pauli_string, cca, qcbm_shots, shots):
    return get_exp_helper(expectation_unitary_dict, pauli_string, cca, qcbm_shots, shots)

In [6]:
#CVXpy solutions
X = np.array([[0, 1], [1, 0]])
Z = np.array([[1,0], [0 ,-1]])
Y = np.array([[0, -1j], [1j, 0]])
H = np.kron(Z,Z) + np.kron(np.eye(2), X) + np.kron(X, np.eye(2))
A_2 = np.kron(np.eye(2),Z)
A_1 = np.kron(Y,np.eye(2))
a1 = 0.2
a2 = 0.1
rho = cp.Variable( np.eye(4).shape, hermitian=True)
constraints = [rho >> 0, cp.real(cp.trace(A_1 @ rho)) >= a1,cp.real(cp.trace(A_2 @ rho)) >= a2, cp.trace(rho) ==1]
prob = cp.Problem(cp.Minimize(cp.real(cp.trace(H @ rho))),
                  constraints)
# Solve SDP
prob.solve(verbose=False)
expected = prob.value
print("Aprroximated value:", prob.value)

WARN: A->p (column pointers) not strictly increasing, column 10 empty
WARN: A->p (column pointers) not strictly increasing, column 15 empty
WARN: A->p (column pointers) not strictly increasing, column 20 empty
WARN: A->p (column pointers) not strictly increasing, column 25 empty
Aprroximated value: -2.2096976087674225


In [24]:
def get_gradient(params, fun):
    delta = 1 - 2 * np.random.binomial(1, 0.5, size=len(params))
    plus = fun(params + perturbation*delta)
    minus = fun(params - perturbation*delta)
    grad = (plus-minus)/(2*perturbation*delta)

    norm = np.linalg.norm(grad)
    if norm > 1:
        grad = grad/norm
    
    return grad

def gradient_descent(fun, x0, jac, maxiter, callback = None):
    x = x0
    for i in range(maxiter):
        iteration = i + 1
        
        x -= lr * jac(x, fun)
        
        if callback != None:
            callback(x, iteration)
    return x

# Interior Point Method

In [19]:
def cost_function_full_IP(params):
    """
    params: in the form of [params]
    """
    # Tr[Hρ]
    ansatz.set_params(params)
    obj = 0
    for i in range(len(H_paulis)):
        obj += get_exp(H_paulis[i], ansatz, qcbm_shots, num_shots)
    
    # Tr[A_iρ] - b_i
    penal = 0
    for i in range(len(As_unitary)):
        constraint = get_exp(As[i], ansatz, qcbm_shots, num_shots) - b[i]
        if constraint <= 0:
            penal += INF
        else:
            penal += np.log(constraint)
            
    cost = obj - mu * penal
    return cost, obj, penal

def cost_function_IP(params):
    """
    params: in the form of [params]
    """
    cost, _, _ = cost_function_full_IP(params)
    return cost

In [20]:
print_every = 100
def callback_IP(x, iteration):
    '''
    Callback function called automatically during optimization. Appends loss value, current iteration and 
    prints every iteration with a completion percentage.
    '''
    global mu
    global prev_params
    global lr
    
    fx, obj, penal = cost_function_full_IP(x)
    IP_loss.append(fx)
    IP_penals.append(penal)
    IP_objectives.append(obj)
        
    # if crossed the boundary, reset the parameters and decrease lr
    if abs(fx - IP_loss[-2]) >= 1e4:
        ansatz.set_params(prev_params)
        x = prev_params
        lr = max(lr / 5, lr_min)
        fx, obj, penal = cost_function_full_IP(x)
        IP_loss[-1] = fx
        IP_penals[-1] = penal
        IP_objectives[-1] = obj

    # change the learning rate dynamically
    if fx > IP_loss[-2]:
        lr = max(lr / 2, lr_max)
    else:
        lr = min(lr * 1.5, lr_min)
    
    if(iteration % print_every == 0):
        print(f"Iteration: {iteration}\tCost: {round(fx,4)}\tPenalty: {round(penal, 4)}\tObj: {round(obj, 4)}\tmu: {mu}\tlr: {round(lr, 4)}")
        mu = max(mu * 0.6, 1e-8)
        
    prev_params = copy.deepcopy(x)

In [21]:
def satisfied(cca):
    """
    Initialize ansatz until the constraints are satisfied
    """
    sat = True
    for i in range(len(As_unitary)):
        constraint = get_exp(As[i], cca, qcbm_shots, num_shots) - b[i]
        if constraint < 0:
            sat = False
    return sat

In [22]:
num_shots = 1e12
qcbm_shots = None

max_iter = 5 * 1000
lr_max = 1e-1
lr_min = 1e-8
perturbation = 1e-4

In [25]:
n = 5
record = 0

costs_IP_outer = []
objs_IP_outer = []
penalties_IP_outer = []

for i in range(n):
    ansatz = create_cca("general", sys_size, ref_size, num_layers, qcbm_layers)

    while not satisfied(ansatz):
        seed = np.random.randint(1e5)
        np.random.seed(seed)
        params = np.random.normal(size=ansatz.get_parameter_num()) % (2 * np.pi)
        ansatz.set_params(params)
        init_seed = seed
        
    mu = 1
    lr = 1e-3
    
    init_params = ansatz.get_params()
    prev_params = copy.deepcopy(init_params)
    
    init_cost, init_obj, init_penal = cost_function_full_IP(init_params)
    IP_loss = [init_cost]
    IP_penals = [init_penal]
    IP_objectives = [init_obj]
    
    print("Expected value:", expected)
    print(f"Inital cost: {round(init_cost, 4)}")
    result = gradient_descent(cost_function_IP, init_params, get_gradient, max_iter, callback=callback_IP)
    
    costs_IP_outer.append(IP_loss)
    objs_IP_outer.append(IP_objectives)
    penalties_IP_outer.append(IP_penals)

    if abs(IP_loss[-1] - expected) <= 0.05:
        dir_name = f"Constrained_Ham_CCA_IP/interior_point"
        problem_name = "IP"
        filename = f"Init_{init_seed}"
        save_data_to_file(IP_loss, IP_penals, IP_objectives, problem_name, dir_name, filename)
        record += 1
        
print(f"Number of successful trials: {record}")

Expected value: -2.2096976087674225
Inital cost: 3.312
Iteration: 100	Cost: 1.284	Penalty: -1.4325	Obj: -0.1485	mu: 1	lr: 0.1
Iteration: 200	Cost: -0.2623	Penalty: -0.7695	Obj: -0.724	mu: 0.6	lr: 0.0
Iteration: 300	Cost: -1.1413	Penalty: -1.7222	Obj: -1.7613	mu: 0.36	lr: 0.1
Iteration: 400	Cost: -1.5085	Penalty: -2.286	Obj: -2.0023	mu: 0.216	lr: 0.1
Iteration: 500	Cost: -1.6988	Penalty: -2.7583	Obj: -2.0563	mu: 0.1296	lr: 0.1
Iteration: 600	Cost: -1.8444	Penalty: -3.4883	Obj: -2.1156	mu: 0.07776	lr: 0.0
Iteration: 700	Cost: -1.9297	Penalty: -4.2037	Obj: -2.1258	mu: 0.046655999999999996	lr: 0.1
Iteration: 800	Cost: -2.0252	Penalty: -3.8968	Obj: -2.1343	mu: 0.027993599999999997	lr: 0.1
Iteration: 900	Cost: -2.0718	Penalty: -4.8017	Obj: -2.1524	mu: 0.016796159999999997	lr: 0.0
Iteration: 1000	Cost: -2.104	Penalty: -4.8017	Obj: -2.1524	mu: 0.010077695999999999	lr: 0.0
Iteration: 1100	Cost: -2.1234	Penalty: -4.8017	Obj: -2.1524	mu: 0.006046617599999999	lr: 0.1
Iteration: 1200	Cost: -2.135	P

Iteration: 4300	Cost: -2.0977	Penalty: -5.0746	Obj: -2.0977	mu: 1e-08	lr: 0.1
Iteration: 4400	Cost: -2.0977	Penalty: -5.0746	Obj: -2.0977	mu: 1e-08	lr: 0.1
Iteration: 4500	Cost: -2.0977	Penalty: -5.0746	Obj: -2.0977	mu: 1e-08	lr: 0.1
Iteration: 4600	Cost: -2.0977	Penalty: -5.0747	Obj: -2.0977	mu: 1e-08	lr: 0.1
Iteration: 4700	Cost: -2.0977	Penalty: -5.0746	Obj: -2.0977	mu: 1e-08	lr: 0.0
Iteration: 4800	Cost: -2.0977	Penalty: -5.0746	Obj: -2.0977	mu: 1e-08	lr: 0.0
Iteration: 4900	Cost: -2.0977	Penalty: -5.0747	Obj: -2.0977	mu: 1e-08	lr: 0.0
Iteration: 5000	Cost: -2.0977	Penalty: -5.0747	Obj: -2.0977	mu: 1e-08	lr: 0.0
Expected value: -2.2096976087674225
Inital cost: 3.7464
Iteration: 100	Cost: 1.3597	Penalty: -1.313	Obj: 0.0467	mu: 1	lr: 0.0
Iteration: 200	Cost: -0.2811	Penalty: -1.3421	Obj: -1.0863	mu: 0.6	lr: 0.0
Iteration: 300	Cost: -1.0032	Penalty: -1.3915	Obj: -1.5041	mu: 0.36	lr: 0.1
Iteration: 400	Cost: -1.4372	Penalty: -1.9319	Obj: -1.8545	mu: 0.216	lr: 0.0
Iteration: 500	Cost: -

Iteration: 3500	Cost: -2.1213	Penalty: -4.2778	Obj: -2.1213	mu: 2.8651179995807016e-08	lr: 0.1
Iteration: 3600	Cost: -2.1213	Penalty: -4.2778	Obj: -2.1213	mu: 1.7190707997484207e-08	lr: 0.1
Iteration: 3700	Cost: -2.1213	Penalty: -4.2778	Obj: -2.1213	mu: 1.0314424798490525e-08	lr: 0.0
Iteration: 3800	Cost: -2.1213	Penalty: -4.2778	Obj: -2.1213	mu: 1e-08	lr: 0.1
Iteration: 3900	Cost: -2.1213	Penalty: -4.2778	Obj: -2.1213	mu: 1e-08	lr: 0.0
Iteration: 4000	Cost: -2.1213	Penalty: -4.2778	Obj: -2.1213	mu: 1e-08	lr: 0.1
Iteration: 4100	Cost: -2.1213	Penalty: -4.2778	Obj: -2.1213	mu: 1e-08	lr: 0.1
Iteration: 4200	Cost: -2.1213	Penalty: -4.2778	Obj: -2.1213	mu: 1e-08	lr: 0.0
Iteration: 4300	Cost: -2.1213	Penalty: -4.2778	Obj: -2.1213	mu: 1e-08	lr: 0.0
Iteration: 4400	Cost: -2.1213	Penalty: -4.2778	Obj: -2.1213	mu: 1e-08	lr: 0.1
Iteration: 4500	Cost: -2.1213	Penalty: -4.2778	Obj: -2.1213	mu: 1e-08	lr: 0.0
Iteration: 4600	Cost: -2.1213	Penalty: -4.2778	Obj: -2.1213	mu: 1e-08	lr: 0.1
Iteration: 47

# QSlack Method

In [26]:
#Problem setup
pauli = ['ZZ','XI','IX','YI','IZ']
H_paulis = ['ZZ','XI','IX']
expectation_unitary_dict = {pauli_string: get_expectation_unitary(pauli_string) for pauli_string in pauli}

a1 = 0.2
a2 = 0.1

In [27]:
def get_trace(ansatz1, ansatz2, qcbm_shots, shots):
    return get_trace_helper(swap_test_unitary, ansatz1, ansatz2, qcbm_shots, shots)

def get_exp(pauli_string, ansatz, qcbm_shots, shots):
    return get_exp_helper(expectation_unitary_dict, pauli_string, ansatz, qcbm_shots, shots)

In [28]:
# precompute terms
# Tr[H^2]=12, Tr[A1^2]=Tr[A2^2]=4
TrH2 = 12
TrA_12 = 4
TrA_22 = 4

In [29]:
def cost_function_full_QS(params):
    # update parameters
    lmbda = params[-2] ** 2
    mu = params[-1] ** 2
     
    params = params[:primal_num_params]
    omega.set_params(params)
  
    # calculate new cost
    obj = 0
    for i in range(len(H_paulis)):
        obj += get_exp(H_paulis[i], omega, qcbm_shots, num_shots)

    est2 = get_exp(pauli[3], omega, qcbm_shots, num_shots)
    est3 = get_exp(pauli[4], omega, qcbm_shots, num_shots)
    penalty = (lmbda - est2 + a1)**2 + (mu - est3 + a2)**2
    cost = obj + c * penalty
    return cost, obj, penalty

def cost_function_QS(params):
    cost, _, _ = cost_function_full_QS(params)
    return cost

In [30]:
print_every = 100
def callback_QS(nfev, x, fx, stepsize, accepted=False):
    '''
    Callback function called automatically during optimization. 
    Appends loss value, penalties, and objective value.
    Print out the training status throughout the training.
    '''
    global primal_loss
    global primal_penals
    global primal_ojectives
    
    fx, obj, penal = cost_function_full_QS(x)
    primal_loss.append(fx)
    primal_penals.append(penal)
    primal_objectives.append(obj)
        
    if(nfev % print_every == 0):
        print(f"Iteration: {nfev//3}\tCost: {round(fx,4)}\tObj: {round(obj, 4)}\tPenalty: {round(penal, 4)}\tc: {c}")

In [31]:
num_shots = 1e12
max_iter = 5000

lr_max = 1e-1
lr_min = 1e-3
dec_interval = 1000
train_lr = get_lr(lr_max, lr_min, dec_interval, max_iter) # halve the lr every 1000 iterations

pertb = 1e-4
trust = True

primal_u_num_params = get_parameter_number("general", sys_size, ref_size, num_layers)
primal_qcbm_num_params = get_parameter_number("qcbm", sys_size, ref_size, qcbm_layers)
primal_num_params = primal_u_num_params + primal_qcbm_num_params

In [32]:
num_runs = 5
record = 0 # record the number of successful trails

costs_primal_outer = []
objs_primal_outer = []
penalties_primal_outer = []

for i in range(num_runs):
    np.random.seed(i)
    lmbda = 0.1
    mu = 0.5
    c = 100

    lr = lr_max
    
    primal_loss = []
    primal_penals = []
    primal_objectives = []

    print(f"Exact value: {expected}")
    omega = create_cca("general", sys_size, ref_size, num_layers, qcbm_layers)
    init_params = [*omega.get_params(), np.sqrt(lmbda), np.sqrt(mu)]
    print(f"Iteration {i}, Initial cost : {cost_function_QS(init_params)}")
    
    autospsa = SPSA(maxiter=max_iter, learning_rate=train_lr,
                        perturbation=pertb, callback=callback_QS, trust_region=trust)
    obj = autospsa.minimize(fun=cost_function_QS, x0=init_params)
    
    costs_primal_outer.append(primal_loss)
    objs_primal_outer.append(primal_objectives)
    penalties_primal_outer.append(primal_penals)

    if abs(primal_loss[-1] - expected) <= 0.05:
        dir_name = f"Constrained_Ham_CCA_IP/QSlack"
        problem_name = "QS"
        filename = f"Init_{i}"
        save_data_to_file(primal_loss, primal_penals, primal_objectives, problem_name, dir_name, filename)
        record += 1
print(f"Number of successful runs: {record}")


Exact value: -2.2096976087674225
Iteration 0, Initial cost : 19.79065884462997
Iteration: 100	Cost: -0.3512	Obj: -0.4142	Penalty: 0.0006	c: 100
Iteration: 200	Cost: -0.7168	Obj: -0.812	Penalty: 0.001	c: 100
Iteration: 300	Cost: -0.8856	Obj: -0.9787	Penalty: 0.0009	c: 100
Iteration: 400	Cost: -0.8106	Obj: -0.9901	Penalty: 0.0018	c: 100
Iteration: 500	Cost: -1.1106	Obj: -1.1689	Penalty: 0.0006	c: 100
Iteration: 600	Cost: -1.2686	Obj: -1.4406	Penalty: 0.0017	c: 100
Iteration: 700	Cost: -1.7	Obj: -1.9055	Penalty: 0.0021	c: 100
Iteration: 800	Cost: -1.7723	Obj: -1.889	Penalty: 0.0012	c: 100
Iteration: 900	Cost: -1.5727	Obj: -1.7792	Penalty: 0.0021	c: 100
Iteration: 1000	Cost: -2.0071	Obj: -2.0761	Penalty: 0.0007	c: 100
Iteration: 1100	Cost: -2.1151	Obj: -2.1244	Penalty: 0.0001	c: 100
Iteration: 1200	Cost: -1.9086	Obj: -2.0396	Penalty: 0.0013	c: 100
Iteration: 1300	Cost: -2.0726	Obj: -2.0862	Penalty: 0.0001	c: 100
Iteration: 1400	Cost: -2.0976	Obj: -2.1314	Penalty: 0.0003	c: 100
Iteration: 1

Iteration: 2300	Cost: -2.1482	Obj: -2.1593	Penalty: 0.0001	c: 100
Iteration: 2400	Cost: -2.1557	Obj: -2.169	Penalty: 0.0001	c: 100
Iteration: 2500	Cost: -2.1571	Obj: -2.1671	Penalty: 0.0001	c: 100
Iteration: 2600	Cost: -2.1632	Obj: -2.1661	Penalty: 0.0	c: 100
Iteration: 2700	Cost: -2.1545	Obj: -2.1572	Penalty: 0.0	c: 100
Iteration: 2800	Cost: -2.1564	Obj: -2.1725	Penalty: 0.0002	c: 100
Iteration: 2900	Cost: -2.1666	Obj: -2.1827	Penalty: 0.0002	c: 100
Iteration: 3000	Cost: -2.1901	Obj: -2.1919	Penalty: 0.0	c: 100
Iteration: 3100	Cost: -2.1919	Obj: -2.1922	Penalty: 0.0	c: 100
Iteration: 3200	Cost: -2.1902	Obj: -2.1904	Penalty: 0.0	c: 100
Iteration: 3300	Cost: -2.1788	Obj: -2.1848	Penalty: 0.0001	c: 100
Iteration: 3400	Cost: -2.1887	Obj: -2.1939	Penalty: 0.0001	c: 100
Iteration: 3500	Cost: -2.1947	Obj: -2.195	Penalty: 0.0	c: 100
Iteration: 3600	Cost: -2.1932	Obj: -2.1951	Penalty: 0.0	c: 100
Iteration: 3700	Cost: -2.1995	Obj: -2.2004	Penalty: 0.0	c: 100
Iteration: 3800	Cost: -2.1988	Obj: -

Iteration: 4700	Cost: -2.2027	Obj: -2.2044	Penalty: 0.0	c: 100
Iteration: 4800	Cost: -2.2021	Obj: -2.2047	Penalty: 0.0	c: 100
Iteration: 4900	Cost: -2.2033	Obj: -2.2043	Penalty: 0.0	c: 100
Iteration: 5000	Cost: -2.2037	Obj: -2.2042	Penalty: 0.0	c: 100
Data saved to Constrained_Ham_CCA_IP/QSlack/Init_4-QS.pkl successfully!
Number of successful runs: 5


# Plots

In [33]:
# plt.rc('font', family='Times New Roman')
plt.rc('font', family='Arial')
plt.rc('axes', titlesize=24)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=14)
plt.rc('legend', framealpha = 1)
plt.rc('legend', shadow = True)

In [34]:
# Size of the smoothing window
window_size_primal = 10
window_size_dual = 10

costs_primal_smooth = get_smooth(costs_primal_outer[:-1], window_size_primal)
costs_IP_smooth = get_smooth(costs_IP_outer[:-1], window_size_dual)

In [35]:
# IQR plot
costs_primal_median = np.median(costs_primal_smooth, axis = 0)
costs_primal_max = np.percentile(costs_primal_smooth, 75, axis = 0)
costs_primal_min = np.percentile(costs_primal_smooth, 25, axis = 0)

costs_IP_median = np.median(costs_IP_smooth, axis = 0)
costs_IP_max = np.percentile(costs_IP_smooth, 75, axis = 0)
costs_IP_min = np.percentile(costs_IP_smooth, 25, axis = 0)

# Plotting the line graph
x1 = len(costs_primal_median)
x2 = len(costs_IP_median)
x3 = max(x1, x2)
expected_line = np.zeros(x3) + expected
plt.plot(np.arange(x1), costs_primal_median, color = 'darkblue', linewidth = 1.5)
plt.plot(np.arange(x2), costs_IP_median, color = 'purple', linewidth = 1.5)
plt.plot(np.arange(x3), expected_line, linestyle='dashed', color = 'deeppink', markersize = 1, linewidth = 2)

plt.legend(['QSlack Method','IP Method', 'True Value'])

plt.fill_between(np.arange(x1), costs_primal_max, costs_primal_min, color = 'darkblue', alpha = .2)
plt.fill_between(np.arange(x2), costs_IP_max, costs_IP_min, color = 'plum', alpha = .2)

plt.xlabel('Iteration')
plt.ylabel('Objective Function Value')
plt.grid(True)
plt.ylim(-3, 5)
plt.xlim(0, max_iter)
# plt.yscale("log")

# Display the graph
plt.show()